In [1]:
import $file.hw4stdlib
import hw4stdlib._

import $file.$        

import hw4stdlib._

# Homework 4

In this assignment we will develop some helpful structures for building the interpreter.Then we will write the first full interpreter for Lettuce. Remember that lettuce is a functional language with let bindings. It is very similiar to a language called ML. Here we want to implement the interpreter for the language from the ground up. We will use most of the existing standard library we have developed while we write this, especially the equality functions and Dictionary data type from earlier in the homework.


## Dictionaries

We will need to define the datatype $\texttt{Dictionary}$ before we write the first interpreter for lettuce. We'll begin by giving the definition below:

$$
\begin{aligned}
\textbf{Dictionary}\ A\ B\ ::&= \text{EmptyDict} \\
&\mid\ \text{Entry}\ A\ B\ (\textbf{Dictionary}\ A\ B)
\end{aligned}
$$

This will create dictionaries of key-value pairs which will be very useful for implementing contexts in the next homework assignment. The definition of this type in Scala is given below:

In [2]:
sealed trait Dictionary[+A, +B] 
case object EmptyDict extends Dictionary[Nothing, Nothing]
case class Entry[A,B](key: A, value: B, rest: Dictionary[A,B]) extends Dictionary[A,B]

defined trait Dictionary
defined object EmptyDict
defined class Entry

For both of the following functions you will need to provide a parameter for an equality-checking function. This is so we can check if keys are equal to eachother

### Membership(In) (5pts)

Write a function called `isIn` which returns `True` if the Dictionary contains a given key and `False` if it does not.

In [3]:
def isIn[A,B](eq_func: (A, A) => Bool, dict: Dictionary[A,B], target: A): Bool = dict match {
    case EmptyDict => False 
    case Entry(key, value, rest) => eq_func(key, target) match {
        case True => True
        case False => isIn(eq_func, rest, target)
    }
}

defined function isIn

In [4]:
assert(isIn(nat_eq, Entry(five, True, EmptyDict), five) == True)
assert(isIn(nat_eq, Entry(eight, True, Entry(five, True, EmptyDict)), five) == True)
assert(isIn(nat_eq, Entry(five, True, EmptyDict), six) == False)
assert(isIn(nat_eq, EmptyDict, seven) == False)
passed(5)


*** Tests Passed (5 points) ***


### Lookup (5 pts)

Write a function that retrieves a value that corresponds to a key in a dictionary. If the key is not in the map then return Nothing.

$$
\texttt{lookup} : (a \rightarrow a \rightarrow \mathbb{B}) \rightarrow \texttt{Dict}\ a\ b \rightarrow a \rightarrow \texttt{Maybe}\ b
$$

In [5]:
def lookup[A,B](eq_func: (A, A) => Bool, dict: Dictionary[A,B], target: A): Maybe[B] = dict match {
    case EmptyDict => None
    case Entry(key, value, rest) => eq_func(key, target) match {
        case True => Just(value)
        case False => lookup(eq_func, rest, target)
    }
}

defined function lookup

In [6]:
assert(lookup(nat_eq, Entry(five, "ham", EmptyDict), five) == Just("ham"))
assert(lookup(nat_eq, Entry(eight, seven, Entry(five, three, EmptyDict)), five) == Just(three))
assert(lookup(nat_eq, Entry(five, True, EmptyDict), six) == None)
assert(lookup(nat_eq, EmptyDict, seven) == None)
passed(5)


*** Tests Passed (5 points) ***


### Practice using these functions (5 pts)
#### Part A
Create the following dictionary as a scala value using this definition:
```
{
    five: True,
    six: False,
    seven: False
}
```
Place it in a variable named "`ans_3a`"

In [7]:
// Solution should look like `val ans3 = ???`
val ans_3a = Entry(five, True, Entry(six, False, Entry(seven, False, EmptyDict)))

ans_3a: Entry[Succ, Product with Bool with Serializable] = Entry(
  key = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero))))),
  value = True,
  rest = Entry(
    key = Succ(
      pred = Succ(
        pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero))))
      )
    ),
    value = False,
    rest = Entry(
      key = Succ(
        pred = Succ(
          pred = Succ(
            pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero))))
          )
        )
      ),
      value = False,
      rest = EmptyDict
    )
  )
)

In [8]:
assert(lookup(nat_eq, ans_3a, five) == Just(True))
assert(lookup(nat_eq, ans_3a, six) == Just(False))
assert(lookup(nat_eq, ans_3a, seven) == Just(False))
passed(2)


*** Tests Passed (2 points) ***


#### Part B
For the following, use the super secret `secret_dict` variable defined below. Use the functions you just defined to complete the following:

1. Look up the value mapped by key `Positive(one)` and place it in variable `ans_3b1` 
2. Look up the value mapped by key `Positive(three)` and place it in variable `ans_3b2`
3. Check whether the key `Negative(three)` is in the map and place True or False in variable `ans_3b3`
4. Check whether the key `Negative(one)` is in the map and place True or False in variable `ans_3b4`


**Note**: For 1 and 2 remember to extract it from the maybe type, set the variable to the string `"does not exist"` if you get `None` instead of `Just(x)`.

In [9]:
val secret_dict = Entry(Positive(three), "abc",
                        Entry(Negative(one), "xyz",
                              Entry(Positive(one), "MIB", EmptyDict)))

secret_dict: Entry[Product with Integer with Serializable, String] = Entry(
  key = Positive(x = Succ(pred = Succ(pred = Succ(pred = Zero)))),
  value = "abc",
  rest = Entry(
    key = Negative(x = Succ(pred = Zero)),
    value = "xyz",
    rest = Entry(
      key = Positive(x = Succ(pred = Zero)),
      value = "MIB",
      rest = EmptyDict
    )
  )
)

In [10]:
// lookup(int_eq, secret_dict, Positive(one))
val ans_3b1 = lookup(int_eq, secret_dict, Positive(one)) match { 
    case None => None
    case Just(x) => x
}
val ans_3b2 = lookup(int_eq, secret_dict, Positive(three)) match { 
    case None => None
    case Just(x) => x
} 

val ans_3b3 = isIn(int_eq, secret_dict, Negative(three))
val ans_3b4 = isIn(int_eq, secret_dict, Negative(one))

ans_3b1: Object with Serializable = "MIB"
ans_3b2: Object with Serializable = "abc"
ans_3b3: Bool = False
ans_3b4: Bool = True

In [10]:
// Hidden tests (3 pts)

# The Lettuce Interpreter



## Problem 1 - The Values (5 points)

Recall that we define interpreters as functions that take in expressions in the form of abstract syntax and give a value as output. In symbols:

$$
eval : Expr \rightarrow Value
$$

It follows that we will need to create a datatype to represent the values that may be computed by lettuce. Bellow is the grammar that defines lettuce values. Implemenet this as a `sealead trait` in Scala below:

$$
\begin{align}
\textbf{Value} ::=&\ NumVal\ \mathbb{Z}\\
\mid&\ BinVal\ \mathbb{B}\\
\mid&\ Error
\end{align}
$$

In [11]:
sealed trait Value
case class NumVal(i: Integer) extends Value
case class BinVal(b: Bool) extends Value
case object Error extends Value

defined trait Value
defined class NumVal
defined class BinVal
defined object Error

If your definition was correct then the values in the cell below should compile.

In [12]:
val v1 = NumVal(Positive(Succ(Succ(Zero))))
val v2 = BinVal(False)
val v3 = Error
passed(5)


*** Tests Passed (5 points) ***


v1: NumVal = NumVal(i = Positive(x = Succ(pred = Succ(pred = Zero))))
v2: BinVal = BinVal(b = False)
v3: Error.type = Error

## Problem 2 - The Expressions (10 points)

There are many possible expressions in the Lettuce language. We also use an abstract syntax to represent this. Here is the grammar to remind you of all the syntactic elements in Lettuce:

$$\begin{array}{rcll}
\mathbf{Expr} & ::= & Const(\mathbb{Z}) \\
 & | & Bin(\mathbb{B}) \\
 & | & Ident(\mathbf{String}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Pow (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Neg (\mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{String}, \mathbf{Expr}, \mathbf{Expr}) \\
\end{array}$$

Now define a `sealed trait` for expressions in Lettuce:

In [13]:
sealed trait Expr
case class Const(i: Integer) extends Expr
case class Bin(b: Bool) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Pow(e1: Expr, e2: Expr) extends Expr
case class Neg(e1: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(s: String, e1: Expr, e2: Expr) extends Expr

defined trait Expr
defined class Const
defined class Bin
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Let

If you defined `Expr` properly then the following expressions should compile:

In [14]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
passed(5)


*** Tests Passed (5 points) ***


ex1: Const = Const(
  i = Positive(
    x = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero)))))
  )
)
ex2: Const = Const(
  i = Negative(x = Succ(pred = Succ(pred = Succ(pred = Zero))))
)
ex3: Bin = Bin(b = True)
ex4: Ident = Ident(s = "x")

In [15]:
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))
passed(5)


*** Tests Passed (5 points) ***


ex5: Plus = Plus(
  e1 = Const(
    i = Positive(
      x = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero)))))
    )
  ),
  e2 = Const(i = Negative(x = Succ(pred = Succ(pred = Succ(pred = Zero)))))
)
ex6: Let = Let(
  s = "x",
  e1 = Plus(
    e1 = Const(
      i = Positive(
        x = Succ(
          pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero))))
        )
      )
    ),
    e2 = Const(i = Negative(x = Succ(pred = Succ(pred = Succ(pred = Zero)))))
  ),
  e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "x"))
)

In [16]:
def maybe_pow(x: Integer, y: Integer): Maybe[Integer]  = y match {
    case Positive(y) => Just(pow(x, y))
    case Negative(y) => None
}
def maybeify_bin = (f: (Integer, Integer) => Integer) => (x: Integer, y: Integer) => Just(f(x, y))
def eval_bin(bin_func: (Integer, Integer) => Maybe[Integer], x: Value, y: Value): Value = 
    (x, y) match {
        case (_, Error) => Error
        case (Error, _) => Error
        case (NumVal(x), NumVal(y)) => bin_func(x, y) match {
            case Just(result) => NumVal(result)
            case None => Error
            case _ => Error
        }
        case _ => Error
    }

defined function maybe_pow
defined function maybeify_bin
defined function eval_bin

In [17]:
def eval[A](locals: Dictionary[String,A], e: Expr): Value = e match {
    case Const(i) => i match {
        case i: Integer => NumVal(i)
        case _ => Error
    }
    case Bin(b) => b match {
        case b: Bool => BinVal(b)
        case _ => Error
    } 
    case Ident(s) => lookup(string_eq, locals, s) match {
        case Just(x) => x match {
            case x: Expr => eval(locals, x) // Keep evaluating until it's resolved to a Value
            case x: Value => x
            case _ => Error
        }
        case None => Error
        case _ => Error
    }
    case Plus(e1, e2) => eval_bin(maybeify_bin(plus), eval(locals, e1), eval(locals, e2)) 
    case Minus(e1, e2) => eval_bin(maybeify_bin(minus), eval(locals, e1), eval(locals, e2))
    case Mult(e1, e2) => eval_bin(maybeify_bin(mult), eval(locals, e1), eval(locals, e2))
    case Pow(e1, e2) => eval_bin(maybe_pow, eval(locals, e1), eval(locals, e2))
    case Let(s, e1, e2) => eval(locals, e1) match {
        case Error => Error
        case v1: Value => eval(Entry(s, e1, locals), e2) 
        case _ => Error
    }
    case Neg(e1) => eval(locals, e1) match {
        case Error => Error
        case NumVal(v1) => NumVal(negate(v1))
        case _ => Error
    }
    case Eq(e1, e2) => (eval(locals, e1), eval(locals, e2)) match {
        case (Error, _) => Error
        case (_, Error) => Error
        case (NumVal(v1), NumVal(v2)) => BinVal(int_eq(v1, v2))
        case (BinVal(v1), BinVal(v2)) => BinVal(bool_eq(v1, v2))
        case (_, _) => Error
    }
    case And(e1, e2) => (eval(locals, e1), eval(locals, e2)) match {
        case (Error, _) => Error
        case (_, Error) => Error
        case (BinVal(v1), BinVal(v2)) => BinVal(and(v1, v2))
        case (_, _) => Error
    }
    case Or(e1, e2) => (eval(locals, e1), eval(locals, e2)) match {
        case (Error, _) => Error
        case (_, Error) => Error
        case (BinVal(v1), BinVal(v2)) => BinVal(or(v1, v2))
        case (_, _) => Error
    }
    case IfThenElse(e1, e2, e3) => eval(locals, e1) match {
        case Error => Error
        case BinVal(predicate) => predicate match {
            case True => eval(locals, e2)
            case False => eval(locals, e3)
        }
        case _ => Error
    }
}

defined function eval

## Problem 3 - The Interpreter (30 pts)

We now need to define the interpreter for this language. It is a function of the form:

$$
eval : Expr \rightarrow Value
$$

Except now that we have an execution environment($\sigma$) we need some way to include sigma in this function. This will be as an additional argument:

$$
eval : \sigma \rightarrow Expr \rightarrow Value
$$

We will let $\sigma$ be a `Dictionary` as we defined in last week's homework. To add something to a dicitonary use the `Entry` constructor to add on a new element. This should be very similiar to the way `Cons` works on lists.

Below we have given you the skeleton of the function `eval`. Fill in each case for the interpreter so that it can interpret all possible Lettuce expressions. The inference rules we covered in the classroom should be very helpful with this.

Hint: It may be helpful to define some helper functions for the inference rules that have similar forms. These are the rules like binary operations for arithmetic, bool, etc. Just like we lumped some of these rules together into a single rule, we can do the same for our interpreter in the form of auxillary functions. The bonus `eval_bin` function from Homework 4 may be a good place for inspiration.

Hint: Be ready to write some nested case matches. Most of your cases should have two nestings. One for the Sytnax element and another for evaluating the expressions it may contain. This is not true for all expressions but most will 

If this was defined correctly then the definitions above should evaluate:

In [18]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))

ex1: Const = Const(
  i = Positive(
    x = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero)))))
  )
)
ex2: Const = Const(
  i = Negative(x = Succ(pred = Succ(pred = Succ(pred = Zero))))
)
ex3: Bin = Bin(b = True)
ex4: Ident = Ident(s = "x")
ex5: Plus = Plus(
  e1 = Const(
    i = Positive(
      x = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero)))))
    )
  ),
  e2 = Const(i = Negative(x = Succ(pred = Succ(pred = Succ(pred = Zero)))))
)
ex6: Let = Let(
  s = "x",
  e1 = Plus(
    e1 = Const(
      i = Positive(
        x = Succ(
          pred = Succ(pred = Succ(pred = Succ(pred = Succ(pred = Zero))))
        )
      )
    ),
    e2 = Const(i = Negative(x = Succ(pred = Succ(pred = Succ(pred = Zero)))))
  ),
  e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "x"))
)

In [19]:
// assert(
eval(EmptyDict, ex3) 
// == BinVal(True))
passed(10)


*** Tests Passed (10 points) ***


res18_0: Value = BinVal(b = True)

In [20]:
assert(eval(EmptyDict, ex4) == Error)
passed(10)


*** Tests Passed (10 points) ***


In [21]:
assert(eval(EmptyDict, ex5) == NumVal(Positive(two)))

In [22]:
assert(eval(EmptyDict, ex6) == NumVal(Positive(four)))
passed(10)


*** Tests Passed (10 points) ***


In [23]:
assert(eval(EmptyDict, Neg(Const(Positive(six)))) == NumVal(Negative(six)))

In [24]:
assert(eval(EmptyDict, Eq(Const(Positive(six)), Const(Positive(five)))) == BinVal(False))
assert(eval(EmptyDict, Eq(Const(Positive(six)), Const(Positive(six)))) == BinVal(True))
assert(eval(EmptyDict, Eq(Bin(True), Bin(True))) == BinVal(True))
assert(eval(EmptyDict, Eq(Bin(False), Bin(True))) == BinVal(False))

In [25]:
assert(eval(EmptyDict, And(Bin(True), Bin(False))) == BinVal(False))
assert(eval(EmptyDict, And(Bin(False), Bin(False))) == BinVal(False))
assert(eval(EmptyDict, And(Bin(True), Bin(True))) == BinVal(True))
assert(eval(EmptyDict, And(Const(Positive(one)), Const(Positive(one)))) == Error)

In [26]:
assert(eval(EmptyDict, Or(Bin(True), Bin(False))) == BinVal(True))
assert(eval(EmptyDict, Or(Bin(False), Bin(False))) == BinVal(False))
assert(eval(EmptyDict, Or(Bin(True), Bin(True))) == BinVal(True))
assert(eval(EmptyDict, And(Const(Positive(one)), Bin(False))) == Error)

In [27]:
assert(eval(EmptyDict, IfThenElse(Bin(True), Const(Positive(one)), Const(Negative(one)))) == NumVal(Positive(one)))
assert(eval(EmptyDict, IfThenElse(Bin(False), Const(Positive(one)), Const(Negative(one)))) == NumVal(Negative(one)))
assert(eval(EmptyDict, IfThenElse(Eq(Bin(False), 
                                     Eq(Const(Positive(two)), Const(Positive(two)))), 
                                     Const(Positive(one)), Const(Negative(one)))) == NumVal(Negative(one)))
assert(eval(EmptyDict, IfThenElse(Bin(False), Const(Positive(one)), Const(Negative(one)))) == NumVal(Negative(one)))